In [2]:
from google.colab import drive
# drive.mount('/content/drive')

In [50]:
import io
import pandas as pd
import re
from datetime import timedelta
import numpy as np

In [190]:
path = '/content/train_test_combined_2012.csv'
pathte = '/content/BPI_Challenge_2012-test.csv'
pathtr = '/content/BPI_Challenge_2012-training.csv'

In [373]:
# d = pd.read_csv (path)
# dtr = pd.read_csv (pathtr)
dte = pd.read_csv (pathte)

In [344]:
d = d.drop('Unnamed: 0', axis = 1)

In [345]:
d['event time:timestamp']= pd.to_datetime(d['event time:timestamp'])

d = d.sort_values(by=['case concept:name', 'event time:timestamp'])

d = d.assign(nexttime=(d.groupby('case concept:name')['event time:timestamp'].shift(-1)))
d['tonext'] = d.nexttime - d['event time:timestamp']

d = d.groupby('case concept:name',as_index = False).apply(lambda x: 
  x.reset_index(drop = True)).reset_index().drop('level_0',axis = 1).rename(columns={"level_1": "actionorder"})
d.actionorder = d.actionorder + 1

predictions = d.groupby('actionorder').tonext.mean()

d = d.assign(predtonext=(predictions[d.actionorder].reset_index().tonext))

In [347]:
d['diff1'] = d.tonext - d.predtonext
d['diff2'] = d.diff1.dt.microseconds + d.diff1.dt.seconds * 10**6 + 864 * 10 ** 8 * d.diff1.dt.days
d['diff3'] = d.diff2 ** 2

'MSE seconds', d.diff3.mean() / (10 ** 6), 'mean seconds', d.diff2.mean() / (10 ** 6)

('MSE seconds', 3.9841262108619116e+18, 'mean seconds', -5.384745075527974e-07)

In [348]:
dtr['event time:timestamp']= pd.to_datetime(dtr['event time:timestamp'])

dtr = dtr.sort_values(by=['case concept:name', 'event time:timestamp'])

dtr = dtr.assign(nexttime=(dtr.groupby('case concept:name')['event time:timestamp'].shift(-1)))
dtr['tonext'] = dtr.nexttime - dtr['event time:timestamp']

dtr = dtr.groupby('case concept:name',as_index = False).apply(lambda x: 
  x.reset_index(drop = True)).reset_index().drop('level_0',axis = 1).rename(columns={"level_1": "actionorder"})
dtr.actionorder = dtr.actionorder + 1

predictions = dtr.groupby('actionorder').tonext.mean()

# dtr = dtr.assign(predtonext=(predictions[dtr.actionorder].reset_index().tonext))

In [374]:
dte['event time:timestamp']= pd.to_datetime(dte['event time:timestamp'])

dte = dte.sort_values(by=['case concept:name', 'event time:timestamp'])

dte = dte.assign(nexttime=(dte.groupby('case concept:name')['event time:timestamp'].shift(-1)))
dte['tonext'] = dte.nexttime - dte['event time:timestamp']

dte = dte.groupby('case concept:name',as_index = False).apply(lambda x: 
  x.reset_index(drop = True)).reset_index().drop('level_0',axis = 1).rename(columns={"level_1": "actionorder"})
dte.actionorder = dte.actionorder + 1

# predictions = dte.groupby('actionorder').tonext.mean()

dte = dte.assign(predtonext=(predictions[dte.actionorder].reset_index().tonext))

dte['predtime'] = dte['event time:timestamp'] +  dte.predtonext

In [375]:
dte['diff1'] = dte.tonext - dte.predtonext
dte['diff2'] = dte.diff1.dt.microseconds + dte.diff1.dt.seconds * 10**6 + 864 * 10 ** 8 * dte.diff1.dt.days
dte['diff3'] = dte.diff2 ** 2

In [376]:
'MSE seconds:', dte.diff3.mean() / (10 ** 6), 'mean seconds:', dte.diff2.mean() / (10 ** 6)

('MSE seconds:', 3.73286735627461e+18, 'mean seconds:', -8528.50257298487)

In [377]:
dexp = dte.drop(['actionorder', 'tonext', 'predtonext', 'diff1', 'diff2', 'diff3', 'nexttime'], axis = 1)
dexp2 = dte.drop(['actionorder', 'tonext', 'predtonext',  'diff1', 'diff2', 'diff3'], axis = 1)

In [378]:
dexp2

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp,nexttime,predtime
0,44964012621824,206324,2012-02-03T17:17:11.047+01:00,2500,A_SUBMITTED,COMPLETE,2012-03-02 17:17:11.047,2012-03-02 17:17:11.323,2012-03-02 17:40:42.193799980
1,44964012621825,206324,2012-02-03T17:17:11.047+01:00,2500,A_PARTLYSUBMITTED,COMPLETE,2012-03-02 17:17:11.323,2012-03-02 17:17:42.964,2012-03-07 10:12:32.462567962
2,44964012621826,206324,2012-02-03T17:17:11.047+01:00,2500,A_DECLINED,COMPLETE,2012-03-02 17:17:42.964,NaT,2012-03-05 08:20:50.191977503
3,44968307589120,206327,2012-02-03T17:23:41.949+01:00,6000,A_SUBMITTED,COMPLETE,2012-03-02 17:23:41.949,2012-03-02 17:23:42.504,2012-03-02 17:47:13.095799980
4,44968307589121,206327,2012-02-03T17:23:41.949+01:00,6000,A_PARTLYSUBMITTED,COMPLETE,2012-03-02 17:23:42.504,2012-03-02 17:24:23.379,2012-03-07 10:19:03.643567962
...,...,...,...,...,...,...,...,...,...
47818,56203942035460,214376,2012-02-29T23:51:16.799+01:00,15000,A_DECLINED,COMPLETE,2012-01-03 09:27:37.118,2012-01-03 09:27:41.325,2012-01-08 02:22:58.257567962
47819,56203942035461,214376,2012-02-29T23:51:16.799+01:00,15000,W_Afhandelen leads,COMPLETE,2012-01-03 09:27:41.325,2012-02-29 23:51:16.799,2012-01-06 00:30:48.552977503
47820,56203942035456,214376,2012-02-29T23:51:16.799+01:00,15000,A_SUBMITTED,COMPLETE,2012-02-29 23:51:16.799,2012-02-29 23:51:17.423,2012-03-09 17:09:46.058373441
47821,56203942035457,214376,2012-02-29T23:51:16.799+01:00,15000,A_PARTLYSUBMITTED,COMPLETE,2012-02-29 23:51:17.423,2012-02-29 23:52:01.287,2012-03-01 13:58:31.850780820


In [380]:
dexp.to_csv('test_predicted_time.csv')
dexp2.to_csv('test_predicted_time_extra.csv')

In [416]:
import collections

In [432]:
minutes = {'30m': 1800000000, '1h': 3600000000, '20m': 1200000000, '10m': 600000000, 
           '5m': 300000000, '1m': 60000000, '2h': 7200000000, '1.5h':5400000000, 
           '2.5h': 9000000000, '3h': 10800000000, '5h':18000000000, '1d': 86400000000}
percents = {}
x = len(dte)

In [433]:
for i in minutes:
  n = len(dte[abs(dte.diff2) - minutes[i] < 0])
  percents[i] = 100*n/x

In [434]:
percents

{'1.5h': 5.522447357965833,
 '10m': 0.09618802668172219,
 '1d': 10.114380109988918,
 '1h': 5.505719005499446,
 '1m': 0.006273132174894925,
 '2.5h': 5.562177195073501,
 '20m': 0.48930430964180416,
 '2h': 5.541266754490517,
 '30m': 5.488990653033059,
 '3h': 5.578905547539887,
 '5h': 5.641636869288836,
 '5m': 0.05436714551575602}

In [435]:
dte['diff4'] = abs(dte.diff2)

'mean abs hours:', dte.diff4.mean() / (10 ** 6) / 3600

('mean abs hours:', 185.89521142305165)